In [1]:
import numpy as np
import numpy.linalg as la
import numpy.random as ra

## <center> Solving a linear set of equations involving lower and upper triangular matrices </center>

In the following, define two functions $\texttt{solveU}$ and $\texttt{solveL}$ which takes as input a upper and lower triangular matrix $\mathbf{U}$ and $\mathbf{L}$ respectively, and a column vector $c$, and respectively solves
\begin{equation}
\mathbf{U} x = c, \qquad \mathbf{L} x = c
\end{equation}

In [2]:
def solveU(U,c):
    
    N = len(c)
    x = np.zeros(N)
    for i in range(N):
        x[(N-1)-i] = (1/U[(N-1)-i, (N-1)-i])*(c[(N-1)-i] - np.dot(U[(N-1)-i, N-i:],x[N-i:]))
                                                                     
    return x

def solveL(L,c):
   
    N = len(c)
    x = np.zeros(N)
    for i in range(N):
        x[i] = (1/L[i,i])*(c[i]-np.dot(L[i,:i],x[:i]))
                
    return x

Your code should be able to run the following

In [3]:
N = 20
epsilon = 1E-6
detbound = -20
tests = 10

for _ in range(tests):
    
    # selects a random nonsingular matrix
    # do not use N too high, because
    # random nonsingular matrices
    # become rarer and rarer
    while True:
        A = ra.rand(N,N)
        U = np.triu(A)
        if np.log10(la.det(U)) > detbound:
            break
    
    c = ra.rand(N)
    xU = solveU(U,c)
    errorU = np.dot(U,xU) - c
    
    assert abs(np.dot(errorU, errorU)) < epsilon

In [4]:
N = 20
epsilon = 1E-6
detbound = -20
tests = 10

for _ in range(tests):
    
    # selects a random nonsingular matrix
    # do not use N too high, because
    # random nonsingular matrices
    # become rarer and rarer
    while True:
        A = ra.rand(N,N)
        L = np.tril(A)
        if np.log10(la.det(L)) > detbound:
            break
    
    c = ra.rand(N)
    xL = solveL(L,c)
    
    errorL = np.dot(L,xL) - c
    
    assert abs(np.dot(errorL, errorL))< epsilon

We want the two functions to assume that the inputted matrices are upper and lower triangular matrices, even though they may not be. This is useful later on, when the zeros may actually be floating points which are small - essentially coming from floating point arithmetic

In [5]:
N = 20
epsilon = 1E-6
detbound = -20
tests = 10

for _ in range(tests):
    
    # selects a random nonsingular matrix
    # do not use N too high, because
    # random nonsingular matrices
    # become rarer and rarer
    while True:
        A = ra.rand(N,N)
        U = np.triu(A)
        if np.log10(la.det(U)) > detbound:
            break
    
    c = ra.rand(N)
    xU1 = solveU(U,c)
    xU2 = solveU(A,c)
    errorU = xU1-xU2
    
    assert abs(np.dot(errorU, errorU)) < epsilon

In [6]:
N = 20
epsilon = 1E-6
detbound = -20
tests = 10

for _ in range(tests):
    
    # selects a random nonsingular matrix
    # do not use N too high, because
    # random nonsingular matrices
    # become rarer and rarer
    while True:
        A = ra.rand(N,N)
        L = np.tril(A)
        if np.log10(la.det(L)) > detbound:
            break
    
    c = ra.rand(N)
    xL1 = solveL(L,c)
    xL2 = solveL(A,c)
    errorU = xL1-xL2
    
    assert abs(np.dot(errorL, errorL)) < epsilon

# <center> LU decomposition and Calculating $\ell_n$ and $U^{(n)}$ </center> 

In the following code, let $\texttt{getvn}$ be a function whose inputs is a matrix and an order index $n$, which outputs the column vector above.

In [7]:
def getvn(U,n):
 
    N = len(U)
    v = np.zeros(N)
    for k in range(n,N):
        v[k] = -U[k,n-1]/U[n-1,n-1]
    return v

The function should pass the following test

In [8]:
testmatrix = np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]])
print(testmatrix)

error = 1E-6

assert max(abs(getvn(testmatrix,1)-[0,-5,-9,-13])) < error
assert max(abs(getvn(testmatrix,2)-[0,0,-10/6,-14/6])) < error
assert max(abs(getvn(testmatrix,3)-[0,0,0,-15/11])) < error

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]


The next function $\texttt{getelln}$ takes in an arbitrary matrix $U$ and an order index $n$, which return $\ell_n$. Useful functions here are $\texttt{np.identity(n)}$ which produces a $n\times n$ identity matrix.

In [9]:
def getelln(U,n):
    
    N = len(U)
    vn = getvn(U,n)
    elln = np.zeros((N,N))
    elln[:,n-1] = vn
    elln += np.identity(N)
    return elln

The function should pass the following test

In [10]:
testmatrix = np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]])
print(testmatrix)

error = 1E-6

assert max(abs(getelln(testmatrix,1)[:,0]-[1,-5,-9,-13])) < error
assert max(abs(getelln(testmatrix,2)[:,1]-[0,1,-10/6,-14/6])) < error
assert max(abs(getelln(testmatrix,3)[:,2]-[0,0,1,-15/11])) < error

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]


Now modify the previous code into a function $\texttt{updateU}$, whose input is a matrix $U$ and an order index $n$, which outputs the pair $v^{(n)}$ and $\ell_n U$

In [11]:
def updateU(U,n):
   
    N = len(U)
    vn = getvn(U,n)
    elln = np.zeros((N,N))
    elln[:,n-1] = vn
    elln += np.identity(N)
    Unew = np.dot(elln,U)
    return vn, Unew

The following code should show if $\texttt{updateU}$ results in an upper triangular matrix when applied thrice on a $4 \times 4$ matrix.

In [12]:
U0 = ra.rand(4,4)
print(U0)
v1, U1 = updateU(U0,1)
print(U1)
v2, U2 = updateU(U1,2)
print(U2)
v3, U3 = updateU(U2,3)
print(U3)

[[0.27782358 0.81415896 0.41044915 0.2509971 ]
 [0.7949096  0.74916761 0.25984197 0.76235175]
 [0.63645968 0.5426904  0.68778963 0.83183843]
 [0.9457129  0.78074046 0.48282842 0.01241005]]
[[ 0.27782358  0.81415896  0.41044915  0.2509971 ]
 [ 0.         -1.5803063  -0.91453629  0.04419813]
 [ 0.         -1.32244775 -0.25249895  0.2568349 ]
 [ 0.         -1.99066087 -0.91434263 -0.8419854 ]]
[[ 2.77823575e-01  8.14158957e-01  4.10449151e-01  2.50997104e-01]
 [ 0.00000000e+00 -1.58030630e+00 -9.14536295e-01  4.41981294e-02]
 [ 0.00000000e+00  0.00000000e+00  5.12812471e-01  2.19848574e-01]
 [ 0.00000000e+00  2.22044605e-16  2.37669243e-01 -8.97660353e-01]]
[[ 2.77823575e-01  8.14158957e-01  4.10449151e-01  2.50997104e-01]
 [ 0.00000000e+00 -1.58030630e+00 -9.14536295e-01  4.41981294e-02]
 [ 0.00000000e+00  0.00000000e+00  5.12812471e-01  2.19848574e-01]
 [ 0.00000000e+00  2.22044605e-16  0.00000000e+00 -9.99551877e-01]]


## <center> Calculating $L^{(n)}$ </center>


## <center> Solving linear sets of equations </center>

Now it's finally time to combine everything we know. First, let us define a function $\texttt{getLU}$, whose input is an arbitrary function $\mathbf{A}$ and whose output are two matrices $\mathbf{L}$ and $\mathbf{U}$ which are the LU factorization of $\mathbf{A}$.

Note that for an $N\times N$ matrix, one needs to use $\texttt{updateU}$ $N-1$ times. Also, one can use the vectors outputted by $\texttt{updateU}$ to construct $\mathbf{L}$.

A useful function here is $\texttt{np.copy}$, which lets you copy the values of a matrix, so that mutations on a copied matrix does not affect the original matrix.

In [14]:
def getLU(A):
  
    N = len(A)
    U = np.copy(A)
    L = np.zeros((N,N))
    for i in range(N-1):
        vi,U = updateU(U,i+1)
        L[:,i] = -vi
    L += np.identity(N)
    return L,U

If the factorization is correct, then $\mathbf{L} \mathbf{U} = A$

In [15]:
N = 20
A = ra.rand(N,N)
L,U = getLU(A)

assert la.norm(np.dot(L,U) - A) < 1E-6

Now let's combine everything finally. Given a matrix $\mathbf{A}$ and a constant column vector $c$, we solve for an unknown set of coefficients $x$ constrained by the following set of linear algebraic equations:
\begin{equation}
\mathbf{A} x = c
\end{equation}
We first factorize $\mathbf{A}$ into $\mathbf{L} \mathbf{U}$ and first solve a intermediate set of coefficients $v$
\begin{equation}
\mathbf{L}v = c
\end{equation}
using $\texttt{solveL}$ and then finally $x$
\begin{equation}
\mathbf{U}x = v
\end{equation}
using $\texttt{solveU}$.

Kindly implement this in the following function $\texttt{solveAxc}$, whose inputs are $\mathbf{A}$ and $c$ and whose output is the solution $x$

In [17]:
def solveAxc(A,c):
   
    L,U = getLU(A)
    v = solveL(L,c)
    x = solveU(U,v)
    return x

Your solution should pass the following unit test.

In [18]:
N = 20
A = ra.rand(N,N)
c = ra.rand(N)
x = solveAxc(A,c)

assert la.norm(np.dot(A,x) - c)<1E-12